In [2]:
pip install yfinance


Note: you may need to restart the kernel to use updated packages.


In [18]:
import yfinance as yf
import pandas as pd
import numpy as np



# Getting Financials
cashflow = ticker.cashflow
balance_sheet = ticker.balancesheet
income_statement = ticker.financials

conservate = int(input("")
tgr = 0.025
years = 5

cash = float(balance_sheet.loc["Cash And Cash Equivalents"].iloc[0])
shares = info.get("sharesOutstanding")
currentprice = ticker.history(period="1d")["Close"].iloc[-1]

# Getting Data for Calculate WACC
total_debt = float(balance_sheet.loc["Total Debt"].iloc[0])
market_cap = float(info.get("marketCap"))

interest_expense = abs(float(income_statement.loc["Interest Expense"].iloc[0]))
tax_expense = abs(float(income_statement.loc["Tax Provision"].dropna().iloc[0]))
pretax_income = float(income_statement.loc["Pretax Income"].dropna().iloc[0])

tax_rate = tax_expense / pretax_income
pretax_cost_of_debt = interest_expense / total_debt

# Calculate cost of equity
beta = info.get("beta")

# 10-year US Treasury yield index
ten_year = yf.Ticker("^TNX")
rf_raw = ten_year.history(period="1d")["Close"].iloc[-1]
rf = float(rf_raw / 100)  # 4.05 -> 0.0405 = 4.05%

def get_market_premium(region="AU"):
    if region.upper() == "AU":
        return 0.06
    elif region.upper() == "US":
        return 0.055
    else:
        return 0.06

market_premium = get_market_premium("AU")

def cost_of_equity(beta, risk_free_rate, market_premium):
    return risk_free_rate + beta * market_premium

equity_cost = cost_of_equity(beta, rf, market_premium)

# Calculate WACC
def calculate_WACC(total_debt, market_cap, pretax_cost_of_debt, cost_of_equity, tax_rate):
    weight_of_debt = total_debt / (total_debt + market_cap)
    return (1 - weight_of_debt) * cost_of_equity + weight_of_debt * pretax_cost_of_debt * (1 - tax_rate)

wacc = calculate_WACC(total_debt, market_cap, pretax_cost_of_debt, equity_cost, tax_rate)

# ---------------------------------
# HISTORICAL RATIOS / MARGINS
# ---------------------------------

# Revenue
revenue = income_statement.loc["Total Revenue"].dropna()
revenue = revenue[::-1]  # oldest -> newest
revenue_growth = revenue.pct_change().dropna()
avg_growth = float(revenue_growth.mean())

forecast_years = 5

# EBIT & margin
ebit = income_statement.loc["EBIT"].dropna()
ebit = ebit[::-1]
ebit_margin = ebit / revenue
avg_ebit_margin = float(ebit_margin.mean())

# D&A (% of revenue)
dep_amort = income_statement.loc["Reconciled Depreciation"].dropna()
dep_amort = dep_amort[::-1]
da_percent = dep_amort / revenue
avg_da_percent = float(da_percent.mean())

# Capex (% of revenue)
capex = cashflow.loc["Capital Expenditure"].dropna()
capex = capex[::-1]
capex = -capex  # make it positive
capex_percent = capex / revenue
avg_capex_percent = float(capex_percent.mean())

# Working capital (% of revenue)
wc = balance_sheet.loc["Working Capital"].dropna()
wc = wc[::-1]  # oldest -> newest
wc_percent = wc / revenue
avg_wc_percent = float(wc_percent.mean())


# ---------------------------------
# FCF & VALUATION HELPERS
# ---------------------------------

def calculate_UnleveredFCF(ebiat, da, capex, delta_wc):
    return ebiat + da - capex - delta_wc

def present_value_fcf(cash_flows, discount_rate):
    cash_flows = np.array(cash_flows)
    pv_flows = np.array([
        cf / (1 + discount_rate) ** t
        for t, cf in enumerate(cash_flows, start=1)
    ])
    return pv_flows

def calculate_terminal_value(cashflow, tgr, discount_rate):
    return (cashflow * (1 + tgr)) / (discount_rate - tgr)

def calculate_equity(ev, cash, debt):
    return ev + cash - debt


# ---------------------------------
# MAIN DCF FUNCTION FOR A GIVEN GROWTH RATE
# ---------------------------------

def dcf_share_price(growth_rate):
    # 1) Forecast revenue
    forecast_revenues = []
    rev = revenue.iloc[-1]  # latest actual revenue
    for _ in range(forecast_years):
        rev *= (1 + growth_rate)
        forecast_revenues.append(rev)
    forecast_revenues = np.array(forecast_revenues)

    # 2) Forecast EBIT, taxes, D&A, Capex, WC
    forecast_ebit = forecast_revenues * avg_ebit_margin
    forecast_taxes = forecast_ebit * tax_rate
    forecast_da = forecast_revenues * avg_da_percent
    forecast_capex = forecast_revenues * avg_capex_percent
    forecast_wc = forecast_revenues * avg_wc_percent

    # 3) ΔWC
    last_wc_actual = wc.iloc[-1]
    delta_wc = []
    prev = last_wc_actual
    for wc_val in forecast_wc:
        delta_wc.append(wc_val - prev)
        prev = wc_val
    delta_wc = np.array(delta_wc)

    # 4) FCFs
    forecast_ebiat = forecast_ebit - forecast_taxes
    unlevered_fcf = calculate_UnleveredFCF(forecast_ebiat, forecast_da, forecast_capex, delta_wc)

    # 5) PV of FCF + terminal value
    pv_fcf = present_value_fcf(unlevered_fcf, wacc)
    last_fcf = unlevered_fcf[-1]
    terminal_value = calculate_terminal_value(last_fcf, tgr, wacc)
    terminal_value_pv = terminal_value / (1 + wacc) ** years

    ev = float(pv_fcf.sum() + terminal_value_pv)
    equity_value = calculate_equity(ev, cash, total_debt)
    share_price = equity_value / shares
    return share_price


# ---------------------------------
# SCENARIOS
# ---------------------------------

base_growth = avg_growth
optimistic_growth = avg_growth * 1.25      # +25% growth vs base
conservative_growth = avg_growth * 0.75    # -25% growth vs base

base_price = dcf_share_price(base_growth)
optimistic_price = dcf_share_price(optimistic_growth)
conservative_price = dcf_share_price(conservative_growth)

print(f"Current market price:           ${currentprice:.2f}")
print(f"Conservative DCF price (-25%):  ${conservative_price:.2f}")
print(f"Base DCF price:                 ${base_price:.2f}")
print(f"Optimistic DCF price (+25%):    ${optimistic_price:.2f}")


SyntaxError: invalid syntax (3748279568.py, line 13)

In [31]:
import yfinance as yf
import pandas as pd
import numpy as np

# -----------------------------
# GLOBAL SETTINGS
# -----------------------------
terminal_growth = 0.015      # 1.5% long-run (GDP-ish)
forecast_years   = 10        # 10-year explicit forecast
terminal_year    = 10        # discount TV at year 10
region           = "AU"      # for equity risk premium

# Path to your tickers file: either .xlsx or .csv
TICKER_FILE = "tickers_example.csv"  


# -----------------------------
# MARKET ASSUMPTIONS
# -----------------------------
ten_year = yf.Ticker("^TNX")
rf_raw = ten_year.history(period="1d")["Close"].iloc[-1]   
rf = float(rf_raw / 100.0)                                


def get_market_premium(region="AU"):
    if region.upper() == "AU":
        return 0.06
    elif region.upper() == "US":
        return 0.055
    else:
        return 0.06


market_premium = get_market_premium(region)


# -----------------------------
# HELPER FUNCTIONS
# -----------------------------
def cost_of_equity(beta, risk_free_rate, market_premium):
    return risk_free_rate + beta * market_premium


def calculate_WACC(total_debt, market_cap, pretax_cost_of_debt, cost_of_equity, tax_rate):
    total_debt = max(total_debt, 0.0)
    if total_debt == 0 or np.isnan(total_debt):
        return cost_of_equity  # all equity

    weight_of_debt = total_debt / (total_debt + market_cap)
    return (1 - weight_of_debt) * cost_of_equity + weight_of_debt * pretax_cost_of_debt * (1 - tax_rate)


def present_value_fcf(cash_flows, discount_rate):
    cash_flows = np.array(cash_flows, dtype=float)
    return np.array([
        cf / (1 + discount_rate) ** t
        for t, cf in enumerate(cash_flows, start=1)
    ])


def calculate_terminal_value(last_fcf, g, discount_rate):
    # guard: ensure discount_rate > g
    if discount_rate <= g:
        discount_rate = g + 0.01
    return (last_fcf * (1 + g)) / (discount_rate - g)


def build_growth_path(initial_growth, years, terminal_g, fade_start=5):
    """
    Years 1–fade_start: use initial_growth
    Years fade_start+1 – years: linear fade from initial_growth to terminal_g
    """
    growth_rates = []
    for year in range(1, years + 1):
        if year <= fade_start:
            g = initial_growth
        else:
            # linear interpolation from initial_growth to terminal_g
            t = year - fade_start
            total_fade_years = years - fade_start
            g = initial_growth + (terminal_g - initial_growth) * (t / total_fade_years)
        growth_rates.append(g)
    return np.array(growth_rates)


# -----------------------------
# CORE DCF FOR ONE TICKER + SCENARIO
# -----------------------------
def dcf_share_price_for_ticker(symbol, growth_multiplier=1.0, verbose=False):
    ticker = yf.Ticker(symbol)
    info = ticker.info

    cashflow        = ticker.cashflow
    balance_sheet   = ticker.balancesheet
    income_statement= ticker.financials

    if cashflow is None or balance_sheet is None or income_statement is None:
        raise ValueError("Missing financial data")

    # Current price and basic info
    current_price = ticker.history(period="1d")["Close"].iloc[-1]
    shares = info.get("sharesOutstanding")
    if shares is None or shares == 0:
        raise ValueError("Missing or zero sharesOutstanding")

    name = info.get("longName") or info.get("shortName") or symbol

    # ------------------ WACC COMPONENTS ------------------
    # Cash and debt
    cash = float(balance_sheet.loc["Cash And Cash Equivalents"].dropna().iloc[0])

    if "Total Debt" in balance_sheet.index:
        total_debt = float(balance_sheet.loc["Total Debt"].dropna().iloc[0])
    else:
        total_debt = 0.0

    # Tax and cost of debt
    interest_expense = abs(float(income_statement.loc["Interest Expense"].dropna().iloc[0]))
    tax_expense = abs(float(income_statement.loc["Tax Provision"].dropna().iloc[0]))
    pretax_income = float(income_statement.loc["Pretax Income"].dropna().iloc[0])
    tax_rate = tax_expense / pretax_income if pretax_income != 0 else 0.25

    pretax_cost_of_debt = interest_expense / total_debt if total_debt > 0 else 0.0

    beta = info.get("beta")
    if beta is None or np.isnan(beta):
        raise ValueError("Missing beta")

    equity_cost = cost_of_equity(beta, rf, market_premium)
    market_cap  = float(info.get("marketCap"))

    wacc = calculate_WACC(total_debt, market_cap, pretax_cost_of_debt, equity_cost, tax_rate)
    if verbose:
        print(symbol, "WACC:", wacc, "Tax rate:", tax_rate)

    # ------------------ HISTORICAL SERIES ------------------
    revenue = income_statement.loc["Total Revenue"].dropna()[::-1]  # oldest → newest
    revenue_growth = revenue.pct_change().dropna()

    base_avg_growth = float(revenue_growth.mean())
    scenario_initial_growth = base_avg_growth * growth_multiplier

    # EBIT margin (median of last up to 5 years) – smoother
    ebit = income_statement.loc["EBIT"].dropna()[::-1]
    ebit_margin_series = (ebit / revenue).dropna()
    ebit_margin_median = float(ebit_margin_series.tail(5).median())

    # D&A as % of revenue (median)
    dep_amort = income_statement.loc["Reconciled Depreciation"].dropna()[::-1]
    da_ratio_series = (dep_amort / revenue).dropna()
    da_ratio_median = float(da_ratio_series.tail(5).median())

    # Working capital as % of revenue (median)
    wc = balance_sheet.loc["Working Capital"].dropna()[::-1]
    wc_ratio_series = (wc / revenue).dropna()
    wc_ratio_median = float(wc_ratio_series.tail(5).median())

    # ------------------ FORECAST REVENUE (10 YEARS, FADED GROWTH) ------------------
    growth_path = build_growth_path(
        initial_growth=scenario_initial_growth,
        years=forecast_years,
        terminal_g=terminal_growth,
        fade_start=5
    )

    last_revenue = revenue.iloc[-1]
    forecast_revenues = []
    rev = last_revenue
    for g in growth_path:
        rev *= (1 + g)
        forecast_revenues.append(rev)
    forecast_revenues = np.array(forecast_revenues)

    # ------------------ FORECAST EBIT, TAXES, D&A, CAPEX, ΔWC ------------------
    forecast_ebit = forecast_revenues * ebit_margin_median
    forecast_ebiat = forecast_ebit * (1 - tax_rate)   # EBIT after tax

    # D&A proportional to revenue
    forecast_da = forecast_revenues * da_ratio_median

    # CapEx = D&A + 0.5% of revenue (Growth A)
    forecast_capex = forecast_da + 0.005 * forecast_revenues

    # ΔWC = WC/Sales * ΔRevenue
    rev_series = np.concatenate(([last_revenue], forecast_revenues))
    delta_wc = []
    for t in range(1, len(rev_series)):
        delta_rev_t = rev_series[t] - rev_series[t - 1]
        delta_wc.append(wc_ratio_median * delta_rev_t)
    delta_wc = np.array(delta_wc)

    # ------------------ FREE CASH FLOW ------------------
    unlevered_fcf = forecast_ebiat + forecast_da - forecast_capex - delta_wc

    # ------------------ PRESENT VALUE + TERMINAL VALUE ------------------
    pv_fcf = present_value_fcf(unlevered_fcf, wacc)
    last_fcf = unlevered_fcf[-1]
    tv = calculate_terminal_value(last_fcf, terminal_growth, wacc)
    tv_pv = tv / (1 + wacc) ** terminal_year

    ev = float(pv_fcf.sum() + tv_pv)
    equity_value = ev + cash - total_debt
    dcf_price = equity_value / shares

    return {
        "ticker": symbol,
        "name": name,
        "current_price": current_price,
        "dcf_price": dcf_price,
        "wacc": wacc,
        "base_avg_growth": base_avg_growth
    }


# -----------------------------
# SCENARIO WRAPPER (CONSERVATIVE / BASE / OPTIMISTIC)
# -----------------------------
def dcf_scenarios(symbol, verbose=False):
    """
    growth_multiplier:
      conservative = 0.75
      base         = 1.0
      optimistic   = 1.25
    """
    base   = dcf_share_price_for_ticker(symbol, growth_multiplier=1.0,  verbose=verbose)
    cons   = dcf_share_price_for_ticker(symbol, growth_multiplier=0.75, verbose=verbose)
    optim  = dcf_share_price_for_ticker(symbol, growth_multiplier=1.25, verbose=verbose)

    return {
        "ticker": base["ticker"],
        "name": base["name"],
        "current_price": float(base["current_price"]),
        "conservative_price": float(cons["dcf_price"]),
        "base_price": float(base["dcf_price"]),
        "optimistic_price": float(optim["dcf_price"])
    }


# -----------------------------
# RUN FOR MULTIPLE TICKERS FROM EXCEL/CSV
# -----------------------------
# Try Excel first, fall back to CSV if needed
try:
    if TICKER_FILE.lower().endswith(".xlsx"):
        tickers_df = pd.read_excel(TICKER_FILE)
    else:
        tickers_df = pd.read_csv(TICKER_FILE)
except Exception as e:
    print(f"Error reading {TICKER_FILE}: {e}")
    raise

results = []

for symbol in tickers_df["Ticker"]:
    symbol = str(symbol).strip()
    if not symbol:
        continue
    try:
        res = dcf_scenarios(symbol)
        results.append(res)
        print(f"Done: {symbol}")
    except Exception as e:
        print(f"Error processing {symbol}: {e}")

results_df = pd.DataFrame(results)
results_df
results_df.to_csv("dcf_results.csv", index=False)

Done: COL.AX
Done: BBY
Done: WMT
Done: WOW.AX
Done: TCO0.DE
